<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_MP1_Food_Vision_Big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Milestone Project: Food Vision Big

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import mixed_precision, Model, Sequential
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomHeight, RandomRotation, RandomWidth, RandomZoom
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report

## Setup

In [2]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-darkgrid')

### Constantes

In [3]:
SEMENTE = 2008193

FORMATO_ENTRADA = (224, 224, 3)
ATIVACAO = 'softmax'

PERDA = 'sparse_categorical_crossentropy'
METRICAS = ['accuracy']

# OTIMIZADOR = 'Adam'
APRENDIZADO = 0.001
APRENDIZADO_RESSINTONIZADO = 0.0001

ITERACOES = 5
ITERACOES_RESSINTONIZADAS = ITERACOES + 5

### Funções 

In [4]:
def preprocessar_imagem(imagem, rotulo, tamanho=224, escalonar=False):
    """
    Redimensiona imagem para (tamanho, tamanho) e converte o dtype para float32.

    Args:
        imagem (tensor): Tensor no formato [lote, altura, largura, canais] ou [altura, largura, canais].
        rotulo (int): Rótulo (não será processado).
        tamanho (int): Tamanho em que a imagem será redimensionada.
        escalonar (bool): A imagem será escalonada ou não.
    
    Return:
        [lote, altura, largura, canais] ou [altura, largura, canais] (float32), rotulo
    """

    imagem = tf.image.resize(imagem, [tamanho, tamanho])

    if escalonar:
        imagem = tf.divide(imagem, 255.)

    return tf.cast(imagem, tf.float32), rotulo

## TensorFlow Dataset: Food101

https://www.tensorflow.org/datasets/overview

In [5]:
(dados_treino, dados_teste), dados_info = tfds.load(name='food101',
                                                    split=['train', 'validation'],
                                                    shuffle_files=True,
                                                    as_supervised=True, # Dados em formato tuple (data, label)
                                                    with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteKVY17P/food101-train.tfrecord*...:   0%|          …

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteKVY17P/food101-validation.tfrecord*...:   0%|     …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [6]:
rotulos = dados_info.features['label'].names

### Exemplo

In [7]:
# exemplo = dados_treino.take(1) # Seleção aleatória
# exemplo

In [8]:
# for imagem, rotulo in exemplo:
#     print(f"""
#     Formato da imagem: {imagem.shape}
#     Tipo de dado da imagem: {imagem.dtype}
#     Alcance dos valores da imagem: {tf.reduce_min(imagem)}, {tf.reduce_max(imagem)}
#     Formato do rótulo: {rotulo.shape}
#     Tipo de dado do rótulo: {rotulo.dtype}
#     Rótulo: {rotulo} ({rotulos[rotulo]})
#     """)

# plt.imshow(imagem)
# plt.title(rotulos[rotulo])
# plt.axis(False);

### Preparar e lotear datasets

https://www.tensorflow.org/guide/data_performance

**Best practice summary**

Here is a summary of the best practices for designing performant TensorFlow input pipelines:

* **Use the `prefetch` transformation** to overlap the work of a producer and consumer
* **Parallelize the data reading transformation** using the `interleave` transformation
* **Parallelize the `map` transformation** by setting the `num_parallel_calls` argument
* **Use the `cache` transformation** to cache data in memory during the first epoch
* **Vectorize user-defined functions** passed in to the `map` transformation
* **Reduce memory usage** when applying the `interleave`, `prefetch`, and `shuffle` transformations

In [9]:
# Mapear dados de treino (função de pré-processamento e paralelização).
dados_treino = dados_treino.map(map_func=preprocessar_imagem, num_parallel_calls=tf.data.AUTOTUNE)

# Embaralhar dados de treino.
dados_treino = dados_treino.shuffle(buffer_size=1000)

# Lotear e pré-buscar dados de treino.
dados_treino = dados_treino.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Mapear, lotear e pré-bucar dados de teste.
dados_teste = dados_teste.map(map_func=preprocessar_imagem, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# dados_treino, dados_teste

## Precisão mista

https://www.tensorflow.org/guide/mixed_precision

In [10]:
# Bugado para EfficientNetBX em TensorFlow 2.5+
# `x` and `y` must have the same dtype, got tf.float16 != tf.float32.

# mixed_precision.set_global_policy("mixed_float16")

## Modelo

In [25]:
modelo_base = EfficientNetB3(include_top=False)
modelo_base.trainable = False

entradas = Input(shape=FORMATO_ENTRADA, name='camada_entrada')

expansao_dados = Sequential(name='expansao_dados')

expansao_dados.add(RandomFlip('horizontal'))
expansao_dados.add(RandomHeight(0.2))
expansao_dados.add(RandomRotation(0.2))
expansao_dados.add(RandomWidth(0.2))
expansao_dados.add(RandomZoom(0.2))

# Há um bug na versão 2.8 do TensorFlow que faz necessário forçar o treinamento para que a expansão dos dados funcione.
expandidos = expansao_dados(entradas, training=True)

camadas = modelo_base(expandidos, training=False)
camadas = GlobalAveragePooling2D(name='agrupamento_media_global')(camadas)

saidas = Dense(len(rotulos), activation=ATIVACAO, name='camada_saida')(camadas)

modelo = Model(inputs=entradas, outputs=saidas)

modelo.compile(loss=PERDA,
               optimizer=Adam(learning_rate=APRENDIZADO),
               metrics=METRICAS)

16705208/16705208 [==============================] - 1s 0us/step


### Verificar uso de precisão mista

In [26]:
# for camada in modelo.layers:
#     print(camada.name, camada.trainable, camada.dtype, camada.dtype_policy)

### Ajustar

In [27]:
historico = modelo.fit(dados_treino,
                       epochs=ITERACOES,
                       steps_per_epoch=len(dados_treino),
                    #    validation_data=dados_teste,
                    #    validation_steps=len(dados_teste),
                       verbose=1)

Epoch 1/5
2368/2368 [==============================] - 306s 126ms/step - loss: 2.0997 - accuracy: 0.4939
Epoch 2/5
2368/2368 [==============================] - 280s 118ms/step - loss: 1.6143 - accuracy: 0.5889
Epoch 3/5
2368/2368 [==============================] - 269s 113ms/step - loss: 1.4978 - accuracy: 0.6159
Epoch 4/5
2368/2368 [==============================] - 260s 109ms/step - loss: 1.4273 - accuracy: 0.6293
Epoch 5/5
2368/2368 [==============================] - 254s 106ms/step - loss: 1.3719 - accuracy: 0.6427


In [ ]:
modelo.save('modelo_MP1_base_enb3_HDF5.h5')

### Avaliar

In [29]:
modelo.evaluate(dados_teste)

790/790 [==============================] - 85s 105ms/step - loss: 1.3187 - accuracy: 0.6486


[1.3187193870544434, 0.648594081401825]

## Sintonia fina

In [30]:
for i, camada in enumerate(modelo.layers):
    print(i, camada.name, camada.trainable)

0 camada_entrada True
1 expansao_dados True
2 efficientnetb0 False
3 agrupamento_media_global True
4 camada_saida True


In [31]:
for i, camada in enumerate(modelo.layers[2].layers):
    print(i, camada.name, camada.trainable)

0 input_1 False
1 rescaling_1 False
2 normalization False
3 tf.math.truediv False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze Fa

In [38]:
# for layer in modelo.layers[2].layers[-16:]:
#     layer.trainable = True

In [39]:
# for i, camada in enumerate(modelo.layers[2].layers):
#     print(i, camada.name, camada.trainable)

0 input_1 False
1 rescaling_1 False
2 normalization False
3 tf.math.truediv False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze Fa

In [40]:
# modelo.compile(loss=PERDA,
#                optimizer=Adam(learning_rate=APRENDIZADO_RESSINTONIZADO),
#                metrics=METRICAS)

# historico_ressintonizado = modelo.fit(dados_treino,
#                                       epochs=ITERACOES_RESSINTONIZADAS,
#                                       steps_per_epoch=len(dados_treino),
#                                       initial_epoch=len(historico.epoch),
#                                     #   validation_data=dados_teste,
#                                     #   validation_steps=len(dados_teste),
#                                       verbose=1)

Epoch 6/10
2368/2368 [==============================] - 259s 106ms/step - loss: 1.2618 - accuracy: 0.6700
Epoch 7/10
2368/2368 [==============================] - 246s 103ms/step - loss: 1.2489 - accuracy: 0.6743
Epoch 8/10
2368/2368 [==============================] - 245s 103ms/step - loss: 1.2451 - accuracy: 0.6752
Epoch 9/10
2368/2368 [==============================] - 251s 105ms/step - loss: 1.2293 - accuracy: 0.6765
Epoch 10/10
2368/2368 [==============================] - 244s 102ms/step - loss: 1.2314 - accuracy: 0.6778


### Reavaliar

In [41]:
# modelo.evaluate(dados_teste)

790/790 [==============================] - 81s 100ms/step - loss: 1.2637 - accuracy: 0.6651


[1.26365327835083, 0.6651089191436768]

## Relatório de classificação

In [42]:
# classes_verdadeiras = []

# for imagem, classe in dados_teste.unbatch():
#     classes_verdadeiras.append(classe.numpy().argmax())

# previsoes = modelo.predict(dados_teste, verbose=1)

# classes_previstas = previsoes.argmax(axis=1)

# print(classification_report(y_true=classes_verdadeiras,
#                             y_pred=classes_previstas,
#                             target_names=rotulos))

## Salvar modelo

In [43]:
# modelo.save('modelo_MP1_HDF5.h5')